In [ ]:
### Imports ###

import os, shutil
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
import torchvision.models as models
from torchvision import transforms
from torchvision import datasets
from PIL import Image
import PIL
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
### Connect to a google drive ###

from google.colab import drive
drive.mount('/content/drive')

ROOT = '/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/'

# Data

This section of notebook prepares data for training and testing and predicting purposes

In [ ]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [ ]:
batch_size=32
img_dimensions = 224
num_workers = 6

img_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

img_test_transforms = transforms.Compose([
    transforms.Resize((img_dimensions,img_dimensions)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

img_aug_transforms = transforms.Compose([
    transforms.Resize((img_dimensions, img_dimensions)),
    transforms.ColorJitter(hue=.05, saturation=.05),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225] )
    ])

def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

test_200_path = ROOT + "test_200"
test_200_data = ImageFolderWithPaths(root=test_200_path,transform=img_transforms, is_valid_file=check_image)
torch.save(test_200_data, ROOT + "test_200_with_filepath.pt")
test_200_data = torch.load(ROOT + "test_200_with_filepath.pt")
test_200_loader = torch.utils.data.DataLoader(test_200_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# final_test_data_path = "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/finat_test"
# final_test_data = ImageFolderWithPaths(root=final_test_data_path,transform=img_transforms, is_valid_file=check_image)
# torch.save(final_test_data, "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/test_data_with_filepath.pt")
# final_test_data = torch.load("/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/test_data_with_filepath.pt")

# train_data_path = "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/train/"
# train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
# torch.save(train_data, "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/train_og.pt")
# train_data = torch.load("/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/train_og.pt")

# validation_data_path = "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/validation/"
# validation_data = torchvision.datasets.ImageFolder(root=validation_data_path,transform=img_test_transforms, is_valid_file=check_image)
# torch.save(validation_data, "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/validation_og.pt")
# validation_data = torch.load("/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/validation_og.pt")

# test_data_path = "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/test/"
# test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_test_transforms, is_valid_file=check_image)
# torch.save(test_data, "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/test_og.pt")
# test_data = torch.load("/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/test_og.pt")

# full_data_path = "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/full/"
# full_data = torchvision.datasets.ImageFolder(root=full_data_path,transform=img_test_transforms, is_valid_file=check_image)
# torch.save(full_data, "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/full_data_with_paths.pt")
# full_data = torch.load("/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/full_data_with_paths.pt")


# num_workers = 6
# train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
# validation_data_loader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
# test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
# final_test_data_loader = torch.utils.data.DataLoader(final_test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
# full_data_loader = torch.utils.data.DataLoader(full_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Models

In this section we train different resnet models on a given dataset and save those models

In [ ]:
model_resnet18 = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
model_resnet34 = torch.hub.load('pytorch/vision', 'resnet34', pretrained=True)
model_resnet101 = torch.hub.load('pytorch/vision', 'resnet101', pretrained=True)
model_resnet152 = torch.hub.load('pytorch/vision', 'resnet152', pretrained=True)

In [ ]:
for name, param in model_resnet18.named_parameters():
    if("bn" not in name):
        param.requires_grad = False
        
for name, param in model_resnet34.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

for name, param in model_resnet101.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

for name, param in model_resnet152.named_parameters():
    if("bn" not in name):
        param.requires_grad = False

In [ ]:
# replace the classifier to get two outputs only
num_classes = 2

model_resnet18.fc = nn.Sequential(nn.Linear(model_resnet18.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

model_resnet34.fc = nn.Sequential(nn.Linear(model_resnet34.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

model_resnet101.fc = nn.Sequential(nn.Linear(model_resnet101.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

model_resnet152.fc = nn.Sequential(nn.Linear(model_resnet152.fc.in_features,512),
                                  nn.ReLU(),
                                  nn.Dropout(),
                                  nn.Linear(512, num_classes))

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=5, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
                        
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.4f}, Validation Loss: {:.4f}, accuracy = {:.4f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [ ]:
model_resnet34.to(device)
optimizer = optim.Adam(model_resnet34.parameters(), lr=0.001)
train(model_resnet34, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, validation_data_loader, epochs=2, device=device)

In [ ]:
model_resnet18.to(device)
optimizer = optim.Adam(model_resnet18.parameters(), lr=0.001)
train(model_resnet18, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, validation_data_loader, epochs=2, device=device)

In [ ]:
model_resnet101.to(device)
optimizer = optim.Adam(model_resnet101.parameters(), lr=0.001)
train(model_resnet101, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, validation_data_loader, epochs=2, device=device)

In [ ]:
model_resnet152.to(device)
optimizer = optim.Adam(model_resnet152.parameters(), lr=0.001)
train(model_resnet152, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, validation_data_loader, epochs=10, device=device)

In [ ]:
# torch.save(model_resnet18.state_dict(), "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/state_dict_resnet18.pt")
# torch.save(model_resnet34.state_dict(), "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/state_dict_resnet34.pt")
# torch.save(model_resnet101.state_dict(), "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/state_dict_resnet101.pt")
# torch.save(model_resnet152.state_dict(), "/content/drive/MyDrive/Colab_Notebooks/cats_vs_dogs/state_dict_resnet152.pt")

In [ ]:
### Load all saved models ###

num_classes = 2

resnet18 = torch.hub.load('pytorch/vision', 'resnet18')
resnet18.fc = nn.Sequential(nn.Linear(resnet18.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
resnet18.load_state_dict(torch.load(ROOT + "state_dict_resnet18.pt"))
resnet18.eval()

resnet34 = torch.hub.load('pytorch/vision', 'resnet34')
resnet34.fc = nn.Sequential(nn.Linear(resnet34.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
resnet34.load_state_dict(torch.load(ROOT + "state_dict_resnet34.pt"))
resnet34.eval()

resnet50 = torch.hub.load('pytorch/vision', 'resnet50')
resnet50.fc = nn.Sequential(nn.Linear(resnet50.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
resnet50.load_state_dict(torch.load(ROOT + "state_dict_resnet50.pt"))
resnet50.eval()

resnet101 = torch.hub.load('pytorch/vision', 'resnet101')
resnet101.fc = nn.Sequential(nn.Linear(resnet101.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
resnet101.load_state_dict(torch.load(ROOT + "state_dict_resnet101.pt"))
resnet101.eval()

resnet152 = torch.hub.load('pytorch/vision', 'resnet152')
resnet152.fc = nn.Sequential(nn.Linear(resnet152.fc.in_features,512),nn.ReLU(), nn.Dropout(), nn.Linear(512, num_classes))
resnet152.load_state_dict(torch.load(ROOT + "state_dict_resnet152.pt"))
resnet152.eval()

# Output test results

Here we export the results of our models for predictions on 25000 and 12500 cats and dogs pictures

In [ ]:
models = [resnet34, resnet50, resnet101, resnet152]
filenames = ['200_resnet34.csv', '200_resnet50.csv', '200_resnet101.csv', '200_resnet152.csv']

In [ ]:
# function to export human sorted pandas framework

def export_human_sorted_df(df, filename):
  df['column'] = df.path.str.extract('(\d+)')
  df['column'] = pd.to_numeric(df['column'])
  df.sort_values(by=['column']).to_csv(ROOT + filename)

In [ ]:
# function to check through the tests and save data 
def test_model(models, loader, filepaths):
  for i in range(len(models)):
    model = models[i]
    filepath = filepaths[i]
    model.to(device)
    correct = 0
    total = 0
    pd_result = pd.DataFrame([])
    pd_paths = pd.DataFrame([])
    with torch.no_grad():
        for data in loader:
            images, labels, paths = data[0].to(device), data[1].to(device), list(data[2])
            outputs = model(images)
            pd_result = pd_result.append(outputs.tolist(), ignore_index=True)
            pd_paths = pd_paths.append(paths, ignore_index=True) 
            _, predicted = torch.max(outputs.data, 1)

    # export results
    merge = pd.concat([pd_result, pd_paths], axis=1)
    merge.columns = ['0', '1', 'path']
    for i in merge.index:
      merge.at[i, 'path'] = merge.at[i, 'path'].split('/')[-1]
    export_human_sorted_df(merge, filepath)

In [ ]:
test_model(models, test_200_loader, filenames)

# Average Ensemble 

In this section we tried to experiment and see what we get if we try to ensemble the models thru averaging their outputs

In [ ]:
### Average of resnet18 and resnet34 ###

models_ensemble = [resnet18.to(device), resnet34.to(device)]
correct = 0
total = 0
ensem_result = pd.DataFrame([])
ensem_paths = pd.DataFrame([])
with torch.no_grad():
    for data in test_200_loader:
        images, labels, paths = data[0].to(device), data[1].to(device), list(data[2])
        predictions = [i(images).data for i in models_ensemble]
        avg_predictions = torch.mean(torch.stack(predictions), dim=0)
        ensem_result = ensem_result.append(avg_predictions.tolist(), ignore_index=True)
        ensem_paths = ensem_paths.append(paths, ignore_index=True) 
        _, predicted = torch.max(avg_predictions, 1)

In [ ]:
merged = pd.concat([ensem_result, ensem_paths], axis=1)
merged.columns = ['0', '1', 'path']
for i in merged.index:
  merged.at[i, 'path'] = merged.at[i, 'path'].split('/')[-1]

export_human_sorted_df(merged, '200_resnet_18_34.csv')